Допустим, у нас есть $n$ товаров с заданными стоимостями $v_i$ и массой $w_i$. В сумку убирается $С$ кг. Сколько какого товара взять, чтобы сумма всех стоимостей товаров была наибольшей?

In [25]:
values = [4, 2, 1, 7, 3, 6]
weights = [5, 9, 8, 2, 6, 5]
C = 15
n = 6

Сформулируем задачу:
$$\max\sum v_i x_i$$
$$\sum w_i x_i \leq C $$

Как должна выглядеть задача:
$$\min c^T x$$
$$A x \leq b $$

Получается, что $c=-v$, $A=w^T$, $b=(C)$

In [26]:
import numpy as np

In [27]:
c = - np.array(values)
A = np.array(weights)         #shape = (6,)
A = np.expand_dims(A, 0)      #shape = (1,6)
b = np.array([C])

In [28]:
from scipy.optimize import linprog

In [29]:
linprog(c=c, A_ub=A, b_ub=b)

C:\Users\User\Anaconda3\lib\site-packages\scipy\optimize\_linprog_util.py:188: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  c = np.array(c, dtype=np.float, copy=True).squeeze()
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


     con: array([], dtype=float64)
     fun: -52.500000000030745
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([-2.24886776e-11])
  status: 0
 success: True
       x: array([6.18738521e-14, 1.05853304e-12, 1.21475941e-13, 7.50000000e+00,
       4.00246685e-13, 4.71394154e-13])

In [ ]:
!pip install cvxpy

In [8]:
import cvxpy

In [9]:
x = cvxpy.Variable(shape=n, integer = True)

In [10]:
constraint = (A @ x <= b)
total_value = c * x

C:\Users\User\Anaconda3\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)


In [11]:
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint])

In [12]:
problem.solve()

SolverError: 

                    You need a mixed-integer solver for this model. Refer to the documentation
                        https://www.cvxpy.org/tutorial/advanced/index.html#mixed-integer-programs
                    for discussion on this topic.

                    Quick fix 1: if you install the python package CVXOPT (pip install cvxopt),
                    then CVXPY can use the open-source mixed-integer solver `GLPK`.

                    Quick fix 2: you can explicitly specify solver='ECOS_BB'. This may result
                    in incorrect solutions and is not recommended.
                

In [13]:
x.value

Теперь положительные $x$

In [14]:
x = cvxpy.Variable(shape=n, integer=True)
constraint = (A @ x <= b)
x_positive = (x >= 0)
total_value = c * x
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint, x_positive])
problem.solve()

C:\Users\User\Anaconda3\lib\site-packages\cvxpy\interface\numpy_interface\ndarray_interface.py:47: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if result.dtype in [numpy.complex, numpy.float64]:
C:\Users\User\Anaconda3\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)


SolverError: 

                    You need a mixed-integer solver for this model. Refer to the documentation
                        https://www.cvxpy.org/tutorial/advanced/index.html#mixed-integer-programs
                    for discussion on this topic.

                    Quick fix 1: if you install the python package CVXOPT (pip install cvxopt),
                    then CVXPY can use the open-source mixed-integer solver `GLPK`.

                    Quick fix 2: you can explicitly specify solver='ECOS_BB'. This may result
                    in incorrect solutions and is not recommended.
                

In [20]:
x.value

array([7.01265807e-10, 7.99333027e-10, 3.58703130e-10, 7.00000000e+00,
       4.67143021e-10, 9.34955115e-10])

Теперь $x = 0$ или $1$

In [21]:
x = cvxpy.Variable(shape=n, boolean=True)
constraint = A @ x <= b
x_positive = x >= 0
total_value = c * x
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint, x_positive])
problem.solve()

-17.00000000382157

In [22]:
x.value

array([1.00000000e+00, 2.25474413e-10, 2.07396078e-10, 1.00000000e+00,
       2.24003299e-10, 1.00000000e+00])

In [ ]:
#x - вектор кол-ва перевезенных товаров (сначала из 1-го склада в 3 ТЦ, затем из 2-го склада в 3 ТЦ)
# ограничения-неравенства: x1+x2+x3 <=180, x4+x5+x6 <= 220
# ограничения-равенства: x1+x4=110, x2+x5=150, x3+x6=140
c1=[2, 5, 3, 7, 7, 6]
A_ub1=[[1, 1, 1, 0, 0, 0], [0, 0, 0, 1, 1, 1]]
b_ub1=[180, 220]
A_eq1=[[1, 0, 0, 1, 0, 0], [0, 1, 0, 0, 1, 0], [0, 0, 1, 0, 0, 1]]
b_eq1=[110, 150, 140]
from scipy.optimize import linprog
import numpy as np
result = linprog(c1, A_ub1, b_ub1, A_eq1, b_eq1)
(result.x*np.array(c1)).sum()